# Fractal Movie Generator #
### -file for Fractal calculations- ###

JV - 2023

'Generates' Fractal movie by using openCV  
Avoiding to use Matplotlib functions due to slow speed and high memory demand  
CMAP_BUF : color space  : hint: keep iteration same size as color space lenght, ie 512  
FRAC: fractal array of floats with values between 0 and 1  
FOI : Field of Interest plot, marks points of interest where to zoom in (max unique points)
IMAGE_BUF: Image buffer, definition kept out of functions to speed up . 

Using @Jit NUMBA compiler options for CPU acceleration  
Hint: check your OpenCV video possibilities, this depends on your installed system (Raspi / Windowsa / Linux etc)
All coordinates are used in Y-X (like Row-Column) to avoid messing up your intepretations between graph and arrays :)

In [2]:
import os 
import cv2
from datetime import datetime
import numba 
from numba import jit
import numpy as np
import scipy as sc
import math
from numpy.random import default_rng
rng = default_rng()
import cmath
from numba import cuda
cuda.select_device(0)
global IMAGENR ; IMAGENR=100
global THREADS ; THREADS=720;
global Gf , Rf, Bf, Zf, If
Gf=2.0;Rf=2.0;Bf=2.0;Zf=0.0;If=0
global COLOR_DEPTH
COLOR_DEPTH=255

In [3]:
# Fractal kernal routines for Mandelbrot
def calculate_mandelbrot_imageC(Ny,Nx,itmax,center,scl,IMG) :                 # Callable function
    x1=center[1]-Nx/(2*scl); x2=center[1]+Nx/(2*scl)
    y1=center[0]-Ny/(2*scl); y2=center[0]+Ny/(2*scl)
   
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp)
    grx=math.ceil(Nx/blx); gry=math.ceil(Ny/bly)
    blockdim = (blx, bly) ; griddim = (grx,gry)
    d_image = cuda.to_device(IMG)
    mandel_kernel[griddim, blockdim](x1, x2, y1, y2, d_image, itmax)
    IMG = d_image.copy_to_host()
    return IMG

@cuda.jit(device=True)
def mandel(x, y, max):
    c = complex(y, x)
    z = 0.0j
    for i in range(max):
        #z = z*z*z*c - z*z*c +z*z + c
        #z = z*z*c + z*c + c
        #z = (z-c)*(z-1) + z*c
        #z = z*z*z + c
        #z = (z-1)*(z-1) + z - c
        #z = z*z/(z-1) + z/(z-1) + c    
        z = (z*z) + c     #orginal manderbrot   
        if (z.real*z.real + z.imag*z.imag) >= 4:
            return i
    return max
       
@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
  startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    real = min_x + (x * pixel_size_x);
    for y in range(startY, height, gridY):
      imag = min_y + (y * pixel_size_y);
      image[y, x] = mandel(real, imag, iters)/iters;
        

# Field of interest calculator : edge detector in 5x5 pixel grid
def calculate_foiC(M,foi):                                            # Callable function
    hy = M.shape[0]
    wx = M.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp)         # optimum is max 512 threads per Block : gtx1080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly)  # calculate grid
    blockdim = (blx, bly) ; griddim = (grx,gry)   # define block and grid
    m_image = cuda.to_device(M)
    f_image = cuda.to_device(foi)
    foi_kernel[griddim, blockdim](m_image,f_image)
    foi=f_image.copy_to_host()
    #M=m_image.copy_to_host()
    return foi   

@cuda.jit
def foi_kernel(fimagein, fimageout):
  height = fimagein.shape[0]
  width = fimagein.shape[1]

  startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
  startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    for y in range(startY, height, gridY):
      p = fimagein[y,x] ;dev=0
      for t in range (-2,3,1) :
        for u in range (-2,3,1) :
            if y+t>=0 and y+t<height and x+u>=0 and x+u<width :
                dev = dev + (p-fimagein[y+t,x+u])*(p-fimagein[y+t,x+u])
      fimageout[y, x] = dev/25  # this should be a square root, but for foi indication the sum^2 is enough
  

# Filter Fractal array
def filter_fracC(fr,cv,fmap) :                                #callable function
    hy = fr.shape[0]
    wx = fr.shape[1]
    cd= fmap.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp); blz=1         # optimum is max 512 threads per Block : gtx1080 , 1024 RTX3080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly); grz= cd  # calculate grid
    blockdim = (blx, bly, blz) ; griddim = (grx,gry,grz)   # define block and grid
    f_image = cuda.to_device(fr)
    c_image = cuda.to_device(cv)
    f_map = cuda.to_device(fmap)
    filter_kernel[griddim, blockdim](f_image,c_image,f_map)
    #cmap=m_image.copy_to_host()
    cv=c_image.copy_to_host()
    #fr=f_image.copy_to_host()    
    return cv
   
    
@cuda.jit
def filter_kernel(frac,cvimg,fmap) :
    hy = frac.shape[0]
    wx = frac.shape[1]
    fy = fmap.shape[0]
    fx = fmap.shape[1]
    tt=0;uu=0;
    startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
    startZ = cuda.blockDim.z * cuda.blockIdx.z + cuda.threadIdx.z
    gridX = cuda.gridDim.x * cuda.blockDim.x;
    gridY = cuda.gridDim.y * cuda.blockDim.y;
    gridZ = cuda.gridDim.z * cuda.blockDim.z;
    for x in range(startX, wx, gridX):
        for y in range(startY, hy, gridY):
          p = frac[y,x]; 
          av=0; wg=0
          if p<0.2 or p>0.80:
            for t in range (0,fy,1) :
                tt = int(t-(fy-1)/2);
                for u in range (0,fx,1) :
                    uu= int(u-(fx-1)/2);
                    if (y+tt)>=0 and (y+tt)<hy and (x+uu)>=0 and (x+uu)<wx :
                          wg = wg + fmap[t,u];
                          av = av + frac[y+tt,x+uu]*fmap[t,u];
            cvimg[y,x] = av/wg  # this should be a square root, but for foi indication the sum^2 is enough
          else:
            cvimg[y,x] = p;
        


# Map fractal array into CV2-image map with R-G-B coding using Color Map
def cmap_cvimageC(fr,cv,cmap) :                                #callable function
    hy = fr.shape[0]
    wx = fr.shape[1]
    cd= cmap.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp); blz=1         # optimum is max 512 threads per Block : gtx1080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly); grz= cd  # calculate grid
    blockdim = (blx, bly, blz) ; griddim = (grx,gry,grz)   # define block and grid
    f_image = cuda.to_device(fr)
    c_image = cuda.to_device(cv)
    m_image = cuda.to_device(cmap)
    cmap_kernel[griddim, blockdim](f_image,c_image,m_image)
    #cmap=m_image.copy_to_host()
    cv=c_image.copy_to_host()
    #fr=f_image.copy_to_host()   
    return cv
    #return cv2.cvtColor(np.array(cv), cv2.COLOR_RGB2BGR)
    
@cuda.jit
def cmap_kernel(frac,cvimg,cmap) :
    hy = frac.shape[0]
    wx = frac.shape[1]
    rr = cmap.shape[0]
    cd = cmap.shape[1]
    startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
    startZ = cuda.blockDim.z * cuda.blockIdx.z + cuda.threadIdx.z
    gridX = cuda.gridDim.x * cuda.blockDim.x;
    gridY = cuda.gridDim.y * cuda.blockDim.y;
    gridZ = cuda.gridDim.z * cuda.blockDim.z;
    for x in range(startX, wx, gridX):
        for y in range(startY, hy, gridY):
            index = int(frac[y,x]*(rr-1))
            index=index%rr                      #protection to avoid faul indexes
            for z in range(startZ, cd, gridZ):
                cvimg[y,x,z]=cmap[index,z]

                
# calculate zoom point by splitting most interesting point of the FOI into an array, then select point, calculate new center.
def calculate_zoompoint(foi,centero,resy,resx, scale, slce) :
    foi_r = np.where(foi > np.max(foi)*0.3 )  # results in 2 1 dimensionla arrays  col:row
    s=np.array([0])
    if slce == 0 : s = np.random.uniform(0,foi_r[:][0].size-1,2)
    else : s[0]=  foi_r[:][0].size*(slce%1)
    xindex=foi_r[1][int(s[0])]
    yindex=foi_r[0][int(s[0])]
    centern= ( centero[0] + (yindex-resy/2)/scale,  centero[1] + (xindex-resx/2)/scale, yindex,xindex)
    return centern # center results in 4 points: real y/x: -> fractal points and mapped y/x -> indexed points of the array


#  Complex function to generate 16bit ColorMaps for Fractals by sinus variations. 
#  Seed =0 : Grayscale, Seed = odd : white color Seed = even: black color
def create_cmap(buf,seed) :
    global Rf; global Bf; 
    global Gf; global Zf ; global If
    r=0;g=0;b=0;inv=0;
    x=buf.shape[0]
    if seed == 0 :         # seed 0 is Gray scale
        r=4;g=4;b=4;inv=0;
    else : 
        if seed == 99 : # color frequency by keyboard
            #do
            r=Rf
            g=Gf
            b=Bf
            inv=If
        else :
            # other seed is random: 
            r= np.random.uniform(0.85,1.95,1)
            g= np.random.uniform(0.85,1.95,1)
            b= np.random.uniform(0.85,1.95,1)
            Rf=r;Bf=b;Gf=g;If=inv;
    for i in range(0,x,1):
        if i*1*math.pi/x < 1*math.pi/6 or i*1*math.pi/x > 5*math.pi/6  : 
            l1= math.cos(i*3*math.pi/x-math.pi/2)  # lightness mask =  halve 4*sine, cut off at 1
        else :  l1= 0.75+ math.cos(i*3*math.pi/x-math.pi/2)/4  # lightness mask =  halve 4*sine, cut off at 1
        if l1>1 : l1=1

        buf[x-i-1][0]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*r*math.pi/x-Zf*math.pi/2))/2  )) # R Channel
        buf[x-i-1][1]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*g*math.pi/x-Zf*math.pi/2))/2  )) # G Channel
        buf[x-i-1][2]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*b*math.pi/x-Zf*math.pi/2))/2  )) # B Channel
        r=r*1.001;g=g*1.001;b=b*1.001
        
    buf.reshape(x,3)
    return buf               

## Start Conditions

In [8]:
#Start Conditions
iterate_max = 512; center = (-0.25,0, 0,0); scale = 200.0; Ny=520; Nx=960; Zf=0.0; # start parameters for iteration max and centerpoint : Y-X coordinates !!
BASE = np.array(np.zeros((Ny,Nx), dtype=np.float64)) # define array global, not in function
FOI=BASE.copy()
IMAGE_BUF = np.zeros((Ny, Nx,3), dtype = np.uint8)
CMAP_BUF =  np.zeros((iterate_max,3), dtype = np.uint8)
CMAP_BUF = create_cmap(CMAP_BUF,2)
FMAP_BUF =  np.matrix ([
      [0.0, 0.0, 0.2, 0.0, 0.0],
      [0.0, 0.2, 0.5, 0.2, 0.0],
      [0.2, 0.5, 0.0, 0.5, 0.2],
      [0.0, 0.2, 0.5, 0.2, 0.0],
      [0.0, 0.0, 0.2, 0.0, 0.0]],dtype=np.float64)

t= datetime.now()
FRAC = calculate_mandelbrot_imageC(Ny,Nx,iterate_max,center,scale,BASE) # calculate mandelbrot
NFRAC = calculate_mandelbrot_imageC(Ny,Nx,iterate_max,center,scale,BASE) # calculate mandelbrot
FRAC =filter_fracC(NFRAC,FRAC,FMAP_BUF)
t=datetime.now()-t
print(t)
C=CMAP_BUF.reshape(1,iterate_max,3)
C=np.repeat(C,32,axis=0)
cv2.imwrite("CMAP.png", C)
frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF)
cv2.imwrite("MandelOrg.png", frame) 
 

C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:215: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  buf[x-i-1][0]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*r*math.pi/x-Zf*math.pi/2))/2  )) # R Channel
C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:216: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  buf[x-i-1][1]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*g*math.pi/x-Zf*math.pi/2))/2  )) # G Channel
C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:217: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensur

0:00:00.758866


True

## Calculate point of interest, 8 iterations

In [11]:
for i in range(0,8,1): # iterations for start - fibnd interesting random area 
        FOI=calculate_foiC(FRAC,FOI)
        center = calculate_zoompoint(FOI,center,Ny,Nx,scale,0)
        scale = scale*1.5;  # zoom in
        FRAC = calculate_mandelbrot_imageC(Ny,Nx,iterate_max,center,scale,FRAC) # calculate mandelbrot
frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF)
cv2.imwrite("Videostart"+str(i)+".png", frame) 
#cv2.imshow('frame',frame)

True

## Stream Video : 7x Panning of 180 zoom steps

In [14]:
frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF) # first FRAC frame
#height, width,c = frame.shape
ncenter=center

# Define the codec and create VideoWriter object.The output is stored in 'outpy.mp4' file.
#out= cv2.VideoWriter('Fractal.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 60, (Nx,Ny))

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('Fractal.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 40, (Nx,Ny))
 
for j in range(0,8,1) :
    FOI=calculate_foiC(FRAC,FOI)
    ncenter = calculate_zoompoint(FOI,center,Ny,Nx,scale,0 )
    #print("Run ",j," - Scale ",scale," - Center ",center[0:2])
    zstep=140
    ystep = (ncenter[0]-center[0])/zstep # panning steps
    xstep = (ncenter[1]-center[1])/zstep # panning steps
    for i in range(0,zstep,1): # iterations 
        cuda.synchronize()
        scale = scale*1.015;   # slowly zoom in
        Zf = (Zf+0.005);
        CMAP_BUF = create_cmap(CMAP_BUF,99)
        center = (center[0] + ystep, center[1] + xstep,0,0); # pan to coordinate
        NFRAC = calculate_mandelbrot_imageC(Ny,Nx,iterate_max,center,scale,BASE) # calculate mandelbrot
        FRAC =filter_fracC(NFRAC,FRAC,FMAP_BUF)
        frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF)
        cv2.imshow('frame2',frame)        
        out.write(frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"): # if the `q` key was pressed, break from the loop
            break        
out.release()
cv2.destroyAllWindows() 

C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:215: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  buf[x-i-1][0]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*r*math.pi/x-Zf*math.pi/2))/2  )) # R Channel
C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:216: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  buf[x-i-1][1]= abs( COLOR_DEPTH*inv -int (COLOR_DEPTH*l1*(1+math.sin(i*g*math.pi/x-Zf*math.pi/2))/2  )) # G Channel
C:\Users\javos\AppData\Local\Temp\ipykernel_45772\1718468096.py:217: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensur